<a href="https://colab.research.google.com/github/HasanJawames/Bears-Polars/blob/main/Copy_of_yolov5ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# select the YOLOv5 version
# VERSION = "v6.1"
VERSION = "v6.0"
#VERSION = "v5.0"

# create the model & config file exporting folder
%mkdir /content/exported_files
INFO_FILE = "/content/exported_files/info.txt"
!touch $INFO_FILE
!echo "YOLOv5 version:" $VERSION >> $INFO_FILE

In [ ]:
# clone YOLOv5 repository
%cd /content
!echo "git clone https://github.com/ultralytics/yolov5" # check the cloning script
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd /content/yolov5
!git log -n 1 # check the latest commit

In [ ]:
%cd /content/yolov5

# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
!pip install -q roboflow
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

#Download Correctly Formatted Custom Dataset

In [ ]:
%cd /content/yolov5

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BkP2lQAHb7P9RP83uyly")
project = rf.workspace("tel-hai-college").project("bears-kind")
dataset = project.version(2).download("yolov5")

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

In [ ]:
# define the epoch number, image size & prefered model file for training
EPOCHS = 20
IMG_SIZE = 500
MODEL = "yolov5s.pt"
MODEL_CONF = "/content/yolov5/models/yolov5s.yaml"

!echo "Epoch:" $EPOCHS >> $INFO_FILE
!echo "Image size:" $IMG_SIZE >> $INFO_FILE
!echo "Base model:" $MODEL >> $INFO_FILE
!echo "Base model config file:" $MODEL_CONF >> $INFO_FILE

#Define Model Configuration and Architecture

In [ ]:
#this is the model configuration we will use for our tutorial
%cat $MODEL_CONF

In [ ]:
# copy the custom YAML file
%cp $MODEL_CONF /content/yolov5/models/custom_model.yaml


# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:

with open('/content/yolov5/models/custom_model.yaml' , 'r') as f:

    #read file
    file_source = f.read()

    #replace 'nc:' with the num_classes in the file
    new_string = 'nc: '+str(num_classes)+' #'
    replace_string = file_source.replace('nc:', new_string)

with open('/content/yolov5/models/custom_model.yaml', 'w') as f:
    #save output
    f.write(replace_string)

%cat /content/yolov5/models/custom_model.yaml

In [ ]:
# copy the config file to the "exported_files" folder
%cp /content/yolov5/models/custom_model.yaml /content/exported_files
%cp {dataset.location}/data.yaml /content/exported_files

#Train Custom YOLOv5 Detector

In [ ]:
# train the "MODEL" on custom data for "EPOCHS" epochs
# you can increase-decrease the batch size
# time its performance
%%time
%cd /content/yolov5/

# !python train.py --img $IMG_SIZE --batch 16 --epochs $EPOCHS --data {dataset.location}/data.yaml --cfg ./models/custom_model.yaml --weights $MODEL --name yolov5_results  --cache

!python train.py --img 640 --batch 16 --epochs $EPOCHS --data {dataset.location}/data.yaml --weights yolov5s.pt --name yolov5_result --cache

# copy the best model to the "exported_files" folder
%cp /content/yolov5/runs/train/yolov5_results/weights/best.pt /content/exported_files

#Evaluate Custom YOLOv5 Detector Performance

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason...
%cd /content/yolov5/
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5_result/results.png', width=1000)  # view results.png

#Visualize Our Training Data with Labels

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/train/yolov5_results/*_batch0_labels.jpg'): #assuming JPG
    display(Image(filename=imageName, width=900))
    break

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5_result/train_batch0.jpg', width=900)

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls /content/yolov5/runs/train/yolov5_result/weights

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/yolov5/runs/train/yolov5_result/weights/best.pt --img $IMG_SIZE --conf 0.4 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")